In [1]:
# Human's emotion recognition using thermal images
import numpy as np
import glob
import random
from numpy.linalg import svd as svd
from configparser import ConfigParser

np.set_printoptions(threshold=np.inf)

# Load db config:
dbconfig = ConfigParser()
dbconfig.read('database.ini')

# Assign dbDir with local config:
dbDir = dbconfig['LOCAL']['location'] #Database directory
eDir = {
    "angry":"anger",
    "fear":"fear",
    "happy":"happy",
    "sad":"sadness",
    "neutral":"neutral"
} #Name of directory of each Emotion

In [5]:
def getSVXFiles(fd):
    return glob.glob(fd + '/*.svx')

def readSVX(file, frame_limit = 6990):
    # Default frame_limit is based on the requirement of at least 1GB of free RAM (6990B*153600B ~~ 1GB)
    
    f = open(file,'rb') #Open file to read
    f.seek(128) #Skip 128bytes
    
    [CA,CB] = np.divide(np.frombuffer(f.read(8), dtype=np.dtype('<i4'), count=2), 2**16)
    #Array from buffer
    #Data type: int32 (4 bytes/each - 'i4') 
    #Byte-order:  Litte-endian ('<')
    
    f.seek(1104) #Move to first frame to read
    
    list_img2d = []
    
    for i in range (0,frame_limit):
        buffer = f.read(153600)
        if len(buffer) < 153600:
            break
        img = np.frombuffer(buffer,dtype=np.dtype('>i2'))
        #Array from buffer
        #Data type: int16 (2 bytes/each - 'i2') 
        #Byte-order:  Big-endian ('>')
        img = img.dot(CB) + CA

        img2d = img.reshape(240,320) #Reshape 1 Dimension to 2 Dimensions for each frame
        list_img2d.append(img2d) #Add frame to array of frames
        
        f.seek(2592,1) #Move to next frame to read
    
    f.close()
    return np.array(list_img2d)


In [4]:
# Demo reading imgs from SVX Files:
import resource
# train = []
for file in getSVXFiles(dbDir+eDir['angry']):
    data = readSVX(file)
    print(file, 'frames = ', len(data))
#     train.extend(data)
    data = None
    print('Total memory allocated: ',resource.getrusage(resource.RUSAGE_SELF).ru_maxrss,'KB')

i:\Thermal Database Ver2.0\anger\anan1.svx frames =  3170
i:\Thermal Database Ver2.0\anger\chinha2.SVX frames =  3304
i:\Thermal Database Ver2.0\anger\cuonga1.svx frames =  3143
i:\Thermal Database Ver2.0\anger\khanha2.SVX frames =  3134
i:\Thermal Database Ver2.0\anger\LamA1.svx frames =  3117
i:\Thermal Database Ver2.0\anger\maya1vx.SVX frames =  3167
i:\Thermal Database Ver2.0\anger\Nama1.svx frames =  3213
i:\Thermal Database Ver2.0\anger\nguyetanger1.SVX frames =  3181
i:\Thermal Database Ver2.0\anger\nhata1.svx frames =  3194
i:\Thermal Database Ver2.0\anger\Nhatangry1.svx frames =  3160
i:\Thermal Database Ver2.0\anger\quana1.svx frames =  3250
i:\Thermal Database Ver2.0\anger\thaoa1.SVX frames =  3258
i:\Thermal Database Ver2.0\anger\thoa1.SVX frames =  3157
i:\Thermal Database Ver2.0\anger\vieta1.svx frames =  3166
